<a href="https://colab.research.google.com/github/EnesGokceDS/Amazon_Reviews_NLP_Capstone_Project/blob/master/Topic_Modeling_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Topic Modeling**
## **Introduction**






Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide

 

> (1) a document-term matrix and

> (2) the number of topics you would like the algorithm to pick up.






Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.


**Importing the data**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls './drive/My Drive/MachineLearning/NLP_capstone_data'

Amazon_logo.PNG			      Amazon_reviews_processed.csv
Amazon_reviews_after_sentiment.csv    Reviews.csv
Amazon_reviews_cleaned_finalform.csv


In [0]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./drive/My Drive/MachineLearning/NLP_capstone_data/Amazon_reviews_after_sentiment.csv')
df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,numerics,upper,word_count,char_count,avg_word,Good_reviews,polarity,subjectivity
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,2011-04-27,good quality dog food,bought several vitality canned dog food produc...,21,3,0,1,0,164,6.2,1,0.42,0.40
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,2012-09-07,not as advertised,product arrived labeled jumbo salted peanutsth...,12,7,0,0,0,136,6.6,0,0.22,0.76
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,2008-08-18,delight says it all,confection around centuries light pillowy citr...,42,18,0,2,0,285,6.1,1,0.19,0.55
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,2011-06-13,cough medicine,looking secret ingredient robitussin believe f...,15,5,0,4,0,128,6.2,0,0.15,0.65
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,2012-10-21,great taffy,great taffy great price wide assortment yummy ...,12,5,0,0,0,83,5.5,1,0.46,0.60


In [5]:
df.isnull().sum()

Id                          0
ProductId                   0
UserId                      0
HelpfulnessNumerator        0
HelpfulnessDenominator      0
Score                       0
Time                        0
Summary                   333
Text                        1
stopwords                   0
punctuation                 0
numerics                    0
upper                       0
word_count                  0
char_count                  0
avg_word                    0
Good_reviews                0
polarity                    0
subjectivity                0
dtype: int64

In [0]:
df= df.dropna()

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


#Topic Modeling with Good reviews (Good reviews =1)

### Create document-term matrix


In [8]:
df.shape

(392384, 19)

In [0]:
df_good_reviews= df.loc[df.Good_reviews ==1][0:10000]

In [0]:
# Create document-term matrix
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(df_good_reviews.Text)

In [31]:
data_cv.shape

(10000, 23437)

In [0]:
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = df_good_reviews.index

In [0]:

# Pickle it for later use
import pickle
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")



In [34]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aa,aaaa,aahhhs,ab,aback,abandon,abates,abberline,abbott,abby,abiding,abilities,ability,abilling,abit,able,ablution,aboard,abor,abou,aboulutely,abounds,aboutbr,aboutexpected,abovebr,abr,abroad,absalutely,absence,absent,absentmindedly,absinthe,absoloutely,absolute,absolutely,absolutley,absolutly,absorb,absorbed,absorbs,...,zest,zesty,zetia,zevia,zico,ziggies,ziggiesstuffed,zilch,zin,zinc,zincother,zinfandel,zinfandels,zing,zingbr,zinger,zingerbr,zinging,zip,ziploc,ziplock,zipped,zipper,zippy,zipseal,zito,ziwipeak,zn,zoe,zojirushi,zomg,zoo,zotz,zridge,zs,zucchini,zukebr,zukes,zuppa,îts
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [36]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,0,2,4,5,6,7,8,9,10,11,13,14,15,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,...,13001,13002,13004,13005,13006,13007,13008,13010,13011,13012,13013,13014,13015,13016,13017,13018,13019,13020,13021,13023,13024,13026,13027,13028,13029,13030,13031,13032,13033,13034,13035,13038,13046,13048,13049,13050,13051,13052,13053,13054
aa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aaaa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aahhhs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aback,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [0]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [39]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.020*"coffee" + 0.011*"like" + 0.010*"tea" + 0.010*"flavor" + 0.009*"good" + 0.009*"taste" + 0.007*"one" + 0.007*"great" + 0.006*"use" + 0.006*"cup"'),
 (1,
  '0.010*"great" + 0.009*"like" + 0.009*"good" + 0.007*"love" + 0.007*"food" + 0.007*"one" + 0.007*"product" + 0.005*"taste" + 0.005*"get" + 0.005*"amazon"')]

In [40]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.016*"coffee" + 0.011*"like" + 0.010*"good" + 0.010*"great" + 0.008*"flavor" + 0.008*"taste" + 0.006*"use" + 0.006*"one" + 0.005*"love" + 0.005*"chips"'),
 (1,
  '0.013*"like" + 0.013*"tea" + 0.010*"taste" + 0.010*"good" + 0.009*"coffee" + 0.009*"flavor" + 0.008*"one" + 0.008*"great" + 0.008*"chocolate" + 0.006*"love"'),
 (2,
  '0.012*"food" + 0.010*"product" + 0.009*"great" + 0.007*"one" + 0.007*"love" + 0.007*"amazon" + 0.007*"good" + 0.006*"dog" + 0.006*"price" + 0.006*"get"')]

In [41]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.010*"like" + 0.008*"great" + 0.008*"good" + 0.007*"use" + 0.007*"sauce" + 0.006*"flavor" + 0.006*"taste" + 0.006*"oil" + 0.005*"make" + 0.005*"seeds"'),
 (1,
  '0.017*"food" + 0.008*"dog" + 0.008*"product" + 0.007*"one" + 0.007*"like" + 0.005*"use" + 0.005*"great" + 0.005*"good" + 0.004*"cat" + 0.004*"milk"'),
 (2,
  '0.012*"great" + 0.010*"good" + 0.009*"love" + 0.009*"amazon" + 0.008*"like" + 0.008*"find" + 0.008*"price" + 0.007*"get" + 0.007*"one" + 0.007*"product"'),
 (3,
  '0.029*"coffee" + 0.014*"like" + 0.014*"tea" + 0.012*"taste" + 0.011*"good" + 0.011*"flavor" + 0.008*"one" + 0.008*"cup" + 0.008*"great" + 0.007*"drink"')]

# **Topic Modeling - Attempt #2 (Nouns Only)** 

In [61]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [66]:
data_nouns = pd.DataFrame(df_good_reviews.Text.apply(nouns))
data_nouns

,Text
0,vitality food products quality product stew me...
2,confection centuries pillowy gelatin nuts case...
4,price assortment yummy taffy delivery taffy lo...
5,hair taffy pound bag flavors root beer melon p...
6,saltwater taffy flavors chewy none candies ver...
...,...
13050,review details vanilla halvahbr confection bar...
13051,chocolate thing dad fathers day
13052,walmart product grocery store none accident co...
13053,sodium chloride potassium chloride form msg co...


In [68]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer


# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.Text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aa,aaaa,aahhhs,ab,abandon,abates,abberline,abbott,abby,abilities,ability,abit,ablution,abor,abounds,aboutbr,abr,absence,absent,absinthe,absolute,absolutley,absorb,absorbs,absorbtion,absorption,abundance,abuse,abusea,acacia,academy,acai,acana,accent,accents,accept,acceptation,accepts,access,accessories,...,zap,zeal,zealand,zen,zero,zerobr,zerocalorie,zest,zesty,zetia,zico,ziggies,zilch,zin,zinc,zincother,zinfandel,zinfandels,zing,zingbr,zinger,zingerbr,zip,ziploc,ziplock,zipper,zito,ziwipeak,zn,zoe,zojirushi,zomg,zoo,zotz,zridge,zs,zucchini,zukebr,zukes,zuppa
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [70]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.016*"product" + 0.013*"taste" + 0.011*"flavor" + 0.010*"chocolate" + 0.008*"price" + 0.007*"cookies" + 0.007*"chips" + 0.006*"eat" + 0.006*"use" + 0.006*"store"'),
 (1,
  '0.040*"coffee" + 0.017*"tea" + 0.015*"food" + 0.014*"taste" + 0.013*"flavor" + 0.010*"product" + 0.009*"cup" + 0.009*"water" + 0.009*"price" + 0.008*"dog"')]

In [71]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.034*"food" + 0.020*"product" + 0.013*"dog" + 0.010*"treats" + 0.010*"dogs" + 0.008*"price" + 0.008*"mix" + 0.007*"seeds" + 0.007*"use" + 0.006*"cat"'),
 (1,
  '0.015*"taste" + 0.012*"flavor" + 0.012*"tea" + 0.011*"product" + 0.010*"chips" + 0.010*"cookies" + 0.010*"bag" + 0.009*"price" + 0.008*"order" + 0.007*"snack"'),
 (2,
  '0.056*"coffee" + 0.022*"flavor" + 0.021*"taste" + 0.015*"chocolate" + 0.013*"cup" + 0.012*"water" + 0.011*"tea" + 0.010*"sugar" + 0.010*"product" + 0.009*"use"')]

This is meaningful:


1.   Pet items
2.   Cookies and snacks
3.   Drinkings




In [72]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.029*"product" + 0.017*"price" + 0.014*"chocolate" + 0.012*"cookies" + 0.011*"use" + 0.011*"store" + 0.011*"taste" + 0.009*"flavor" + 0.009*"order" + 0.008*"amazon"'),
 (1,
  '0.016*"milk" + 0.010*"nuts" + 0.010*"peanuts" + 0.009*"candy" + 0.007*"chocolate" + 0.006*"cake" + 0.005*"almonds" + 0.004*"work" + 0.004*"mint" + 0.004*"party"'),
 (2,
  '0.060*"coffee" + 0.026*"tea" + 0.025*"food" + 0.014*"cup" + 0.013*"taste" + 0.012*"flavor" + 0.012*"dog" + 0.010*"treats" + 0.010*"dogs" + 0.008*"use"'),
 (3,
  '0.022*"taste" + 0.020*"flavor" + 0.015*"chips" + 0.014*"sugar" + 0.011*"water" + 0.011*"snack" + 0.010*"flavors" + 0.008*"bag" + 0.008*"juice" + 0.008*"salt"')]

# **Topic Modeling - Attempt #3 (Nouns and Adjectives)**

In [46]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [48]:
data_nouns_adj = pd.DataFrame(df_good_reviews.Text.apply(nouns_adj))
data_nouns_adj

,Text
0,several vitality dog food products good qualit...
2,confection centuries light pillowy gelatin nut...
4,great taffy great price wide assortment yummy ...
5,wild hair taffy pound bag enjoyable many flavo...
6,saltwater taffy great flavors soft chewy none ...
...,...
13050,review details vanilla halvahbr confection bar...
13051,chocolate best thing dad happy shipment father...
13052,walmart product tried find internet grocery st...
13053,sodium chloride potassium chloride free form m...


In [51]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer() #, max_df is used for removing data values that appear too frequently, also known as "corpus-specific stop words". 
# For example, max_df=.8 means "It ignores terms that appear in more than 80% of the documents". I won't use max_df limiter here.
data_cvna = cvna.fit_transform(data_nouns_adj.Text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aa,aaaa,aahhhs,ab,abandon,abates,abberline,abbott,abby,abilities,ability,abit,able,ablution,abor,abounds,aboutbr,abovebr,abr,absence,absent,absinthe,absolute,absolutely,absolutley,absorb,absorbed,absorbs,absorbtion,absorption,abt,abu,abundance,abundant,abuse,abusea,abut,acacia,academy,acai,...,zen,zero,zerobr,zerocalorie,zest,zesty,zetia,zevia,zico,ziggies,zilch,zin,zinc,zincother,zinfandel,zinfandels,zing,zingbr,zinger,zingerbr,zip,ziploc,ziplock,zipper,zippy,zito,ziwipeak,zn,zoe,zojirushi,zomg,zoo,zotz,zridge,zs,zucchini,zukebr,zukes,zuppa,îts
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [57]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.020*"coffee" + 0.015*"great" + 0.015*"good" + 0.011*"food" + 0.010*"product" + 0.008*"price" + 0.007*"taste" + 0.006*"time" + 0.006*"flavor" + 0.006*"amazon"'),
 (1,
  '0.014*"tea" + 0.011*"good" + 0.011*"taste" + 0.010*"flavor" + 0.009*"great" + 0.008*"hot" + 0.008*"chocolate" + 0.007*"sugar" + 0.007*"water" + 0.006*"product"')]

In [58]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.017*"great" + 0.014*"good" + 0.014*"product" + 0.009*"price" + 0.008*"chocolate" + 0.008*"cookies" + 0.008*"taste" + 0.008*"amazon" + 0.006*"order" + 0.006*"time"'),
 (1,
  '0.035*"coffee" + 0.017*"tea" + 0.014*"good" + 0.014*"flavor" + 0.013*"taste" + 0.010*"great" + 0.010*"cup" + 0.008*"water" + 0.006*"use" + 0.006*"hot"'),
 (2,
  '0.018*"food" + 0.012*"great" + 0.011*"good" + 0.008*"dog" + 0.007*"chips" + 0.006*"product" + 0.006*"eat" + 0.006*"bag" + 0.006*"little" + 0.006*"healthy"')]

In [59]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.055*"coffee" + 0.013*"good" + 0.013*"cup" + 0.012*"great" + 0.011*"flavor" + 0.010*"tea" + 0.009*"taste" + 0.007*"product" + 0.006*"use" + 0.006*"price"'),
 (1,
  '0.015*"good" + 0.014*"tea" + 0.013*"taste" + 0.013*"great" + 0.011*"chocolate" + 0.010*"sugar" + 0.010*"flavor" + 0.009*"milk" + 0.009*"water" + 0.008*"product"'),
 (2,
  '0.017*"great" + 0.015*"good" + 0.012*"flavor" + 0.011*"taste" + 0.010*"chips" + 0.008*"snack" + 0.007*"bag" + 0.007*"salt" + 0.006*"delicious" + 0.006*"oil"'),
 (3,
  '0.023*"food" + 0.014*"product" + 0.010*"great" + 0.010*"dog" + 0.010*"good" + 0.008*"price" + 0.007*"time" + 0.007*"treats" + 0.007*"dogs" + 0.006*"old"')]

## **Identify Topics in Each Document** 



In [60]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.046*"coffee" + 0.016*"good" + 0.015*"flavor" + 0.014*"chocolate" + 0.014*"taste" + 0.013*"cup" + 0.012*"great" + 0.008*"hot" + 0.007*"mix" + 0.007*"best"'),
 (1,
  '0.017*"great" + 0.017*"good" + 0.013*"flavor" + 0.012*"taste" + 0.011*"chips" + 0.008*"cookies" + 0.007*"delicious" + 0.007*"snack" + 0.006*"sweet" + 0.006*"salt"'),
 (2,
  '0.025*"food" + 0.008*"juice" + 0.007*"good" + 0.007*"organic" + 0.007*"cat" + 0.007*"seeds" + 0.006*"water" + 0.006*"natural" + 0.005*"foods" + 0.005*"taste"'),
 (3,
  '0.023*"tea" + 0.018*"product" + 0.016*"great" + 0.013*"price" + 0.013*"good" + 0.010*"amazon" + 0.009*"time" + 0.008*"dog" + 0.007*"store" + 0.006*"bag"')]

##### Another final model candidate: Only noun -  3 topic

In [77]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=80)
ldan.print_topics()

[(0,
  '0.053*"coffee" + 0.025*"tea" + 0.020*"taste" + 0.019*"flavor" + 0.013*"cup" + 0.011*"chocolate" + 0.011*"water" + 0.011*"product" + 0.009*"price" + 0.008*"use"'),
 (1,
  '0.034*"food" + 0.016*"product" + 0.013*"dog" + 0.012*"cookies" + 0.010*"dogs" + 0.009*"treats" + 0.008*"price" + 0.008*"mix" + 0.008*"milk" + 0.007*"seeds"'),
 (2,
  '0.016*"flavor" + 0.014*"taste" + 0.014*"chips" + 0.013*"product" + 0.009*"price" + 0.008*"bag" + 0.008*"snack" + 0.007*"order" + 0.007*"store" + 0.007*"salt"')]

similar result with passes 10. Just group orders are different



---



---



---



#Topic Modeling with Bad reviews (Good reviews =0)

In [0]:
df_good_reviews= df.loc[df.Good_reviews ==0][0:10000]

In [0]:
# Create document-term matrix
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(df_good_reviews.Text)

data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = df_good_reviews.index

In [84]:
# Pickle it 
import pickle
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")

# Let's read in our document-term matrix
data = pd.read_pickle('dtm_stop.pkl')
data

,aa,aachen,aadmit,aback,abandoned,abandoning,abbott,abc,abdomen,abdominal,abide,abietatebr,abilities,abilitiesbr,ability,abit,able,abnormalities,abnormally,aboiut,abolutely,abomination,abominations,abound,aboutbr,aboutbut,aboveaverage,abovebr,aboves,abr,abrasive,abrupt,absence,absencebr,absense,absent,absolute,absolutely,absolutelybr,absolutley,...,zico,zicotoo,zilch,zinc,zinczenko,zing,zinger,zingnot,zip,zipfizz,zipfizzbr,ziploc,ziplock,zipp,zipped,zipper,zipperdidnt,zippfizz,zippy,ziptop,zits,ziwipeak,zoji,zojirushi,zola,zombiebr,zombies,zon,zone,zoology,zoom,zooming,zout,zucchini,zucchinibr,zuchon,zuke,zukes,zupas,çelem
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43340,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43372,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,1,3,12,16,26,44,46,48,49,52,59,61,66,67,72,73,74,77,82,83,98,105,109,128,130,143,145,147,152,160,163,166,167,168,181,188,196,199,207,211,...,43139,43140,43144,43145,43146,43156,43169,43170,43196,43201,43204,43208,43209,43210,43214,43231,43232,43256,43262,43265,43267,43270,43281,43289,43299,43306,43308,43309,43318,43319,43320,43321,43325,43328,43339,43340,43369,43372,43373,43376
aa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aadmit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aback,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [0]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [88]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.012*"coffee" + 0.011*"tea" + 0.011*"like" + 0.008*"one" + 0.007*"would" + 0.007*"dog" + 0.006*"taste" + 0.006*"get" + 0.005*"food" + 0.005*"good"'),
 (1,
  '0.013*"like" + 0.012*"taste" + 0.010*"product" + 0.008*"flavor" + 0.008*"good" + 0.007*"would" + 0.007*"chocolate" + 0.006*"one" + 0.005*"much" + 0.005*"dont"')]

In [89]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.012*"product" + 0.009*"one" + 0.008*"dog" + 0.008*"would" + 0.008*"amazon" + 0.007*"get" + 0.006*"box" + 0.006*"like" + 0.005*"time" + 0.005*"food"'),
 (1,
  '0.017*"like" + 0.017*"taste" + 0.010*"flavor" + 0.009*"chocolate" + 0.009*"good" + 0.007*"would" + 0.007*"product" + 0.006*"ginger" + 0.006*"one" + 0.006*"really"'),
 (2,
  '0.025*"coffee" + 0.022*"tea" + 0.013*"like" + 0.008*"taste" + 0.007*"flavor" + 0.007*"cup" + 0.007*"food" + 0.006*"one" + 0.006*"good" + 0.006*"would"')]

In [90]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.012*"product" + 0.009*"food" + 0.009*"one" + 0.008*"dog" + 0.008*"would" + 0.007*"amazon" + 0.006*"get" + 0.005*"like" + 0.005*"box" + 0.005*"time"'),
 (1,
  '0.023*"tea" + 0.017*"like" + 0.015*"taste" + 0.012*"ginger" + 0.012*"drink" + 0.011*"flavor" + 0.008*"water" + 0.007*"would" + 0.007*"good" + 0.006*"sugar"'),
 (2,
  '0.050*"coffee" + 0.011*"like" + 0.010*"cup" + 0.008*"taste" + 0.007*"flavor" + 0.007*"good" + 0.006*"roast" + 0.005*"acid" + 0.005*"one" + 0.005*"would"'),
 (3,
  '0.018*"like" + 0.015*"taste" + 0.012*"chocolate" + 0.009*"good" + 0.009*"flavor" + 0.007*"would" + 0.007*"one" + 0.007*"product" + 0.006*"really" + 0.006*"dont"')]

### **Topic Modeling - Attempt #2 (Nouns Only)** 

In [91]:
data_nouns = pd.DataFrame(df_good_reviews.Text.apply(nouns))
data_nouns

,Text
1,product jumbo peanutsthe peanuts error vendor ...
3,ingredient robitussin addition root beer extra...
12,cats platinum years bag shape food food sit ki...
16,tv movies transfer zip lock baggie stay take time
26,candy flavor plan chewy
...,...
43340,product cocoaantioxidants body surprize order ...
43369,dutch cocoa like method production nothing red...
43372,product dutch process cocoa product dutch proc...
43373,seattle coffeenot wontbr decaf


In [92]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer


# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.Text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aa,aadmit,abbott,abc,abdomen,abietatebr,abilities,ability,abit,abnormalities,abomination,abominations,aboutbr,aboveaverage,aboves,abr,absence,absense,absolute,absolutley,absorb,absorber,absorbs,absorption,abundance,ac,acacia,acai,accelerant,accenton,accents,accentsbr,accentuates,accept,acceptance,access,accident,accompanies,accompaniment,accompany,...,zbones,zeal,zealand,zero,zerobr,zerostar,zest,zesty,zevia,zhenas,zico,zicotoo,zilch,zinc,zing,zinger,zingnot,zip,zipfizz,zipfizzbr,ziploc,ziplock,zipp,zipper,zipperdidnt,zits,ziwipeak,zoji,zojirushi,zola,zombiebr,zombies,zon,zone,zoom,zucchini,zuchon,zuke,zukes,çelem
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43340,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43372,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [94]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.025*"product" + 0.019*"food" + 0.011*"dog" + 0.008*"box" + 0.008*"dogs" + 0.008*"treats" + 0.007*"price" + 0.007*"order" + 0.007*"bag" + 0.006*"eat"'),
 (1,
  '0.036*"taste" + 0.025*"coffee" + 0.025*"flavor" + 0.020*"tea" + 0.015*"chocolate" + 0.013*"product" + 0.011*"ginger" + 0.010*"water" + 0.008*"sugar" + 0.007*"tastes"')]

Prevailing topics


1.   Pet items
2.   Hot drinkings



In [95]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.040*"coffee" + 0.025*"product" + 0.012*"dog" + 0.011*"price" + 0.011*"box" + 0.010*"treats" + 0.010*"order" + 0.009*"dogs" + 0.008*"amazon" + 0.007*"bag"'),
 (1,
  '0.030*"food" + 0.019*"product" + 0.011*"cans" + 0.010*"eat" + 0.008*"ingredients" + 0.008*"cats" + 0.007*"cat" + 0.006*"jerky" + 0.006*"chicken" + 0.005*"dog"'),
 (2,
  '0.043*"taste" + 0.027*"flavor" + 0.026*"tea" + 0.019*"chocolate" + 0.014*"ginger" + 0.013*"product" + 0.011*"water" + 0.010*"sugar" + 0.008*"tastes" + 0.008*"drink"')]

In [96]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.043*"product" + 0.016*"box" + 0.015*"price" + 0.014*"order" + 0.013*"amazon" + 0.011*"cans" + 0.010*"item" + 0.009*"company" + 0.008*"bag" + 0.008*"package"'),
 (1,
  '0.035*"food" + 0.023*"dog" + 0.016*"dogs" + 0.015*"treats" + 0.012*"product" + 0.010*"cats" + 0.009*"eat" + 0.009*"cat" + 0.007*"treat" + 0.007*"jerky"'),
 (2,
  '0.044*"coffee" + 0.039*"taste" + 0.027*"chocolate" + 0.027*"flavor" + 0.008*"cookies" + 0.008*"bar" + 0.008*"cup" + 0.007*"product" + 0.007*"texture" + 0.007*"salt"'),
 (3,
  '0.047*"tea" + 0.035*"taste" + 0.025*"ginger" + 0.024*"flavor" + 0.018*"water" + 0.014*"drink" + 0.013*"sugar" + 0.012*"product" + 0.010*"juice" + 0.008*"tastes"')]

This also has some clear topic:


1.   Service complains
2.   Pet items
3.   Coffee 
4.   Tea 




### **Topic Modeling - Attempt #3 (Nouns and Adjectives)**

In [97]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer() #, max_df is used for removing data values that appear too frequently, also known as "corpus-specific stop words". 
# For example, max_df=.8 means "It ignores terms that appear in more than 80% of the documents". I won't use max_df limiter here.
data_cvna = cvna.fit_transform(data_nouns_adj.Text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aa,aaaa,aahhhs,ab,abandon,abates,abberline,abbott,abby,abilities,ability,abit,able,ablution,abor,abounds,aboutbr,abovebr,abr,absence,absent,absinthe,absolute,absolutely,absolutley,absorb,absorbed,absorbs,absorbtion,absorption,abt,abu,abundance,abundant,abuse,abusea,abut,acacia,academy,acai,...,zen,zero,zerobr,zerocalorie,zest,zesty,zetia,zevia,zico,ziggies,zilch,zin,zinc,zincother,zinfandel,zinfandels,zing,zingbr,zinger,zingerbr,zip,ziploc,ziplock,zipper,zippy,zito,ziwipeak,zn,zoe,zojirushi,zomg,zoo,zotz,zridge,zs,zucchini,zukebr,zukes,zuppa,îts
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [99]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.025*"coffee" + 0.012*"tea" + 0.011*"food" + 0.011*"good" + 0.009*"great" + 0.008*"taste" + 0.007*"flavor" + 0.007*"cup" + 0.006*"product" + 0.006*"dog"'),
 (1,
  '0.016*"great" + 0.016*"good" + 0.010*"product" + 0.009*"taste" + 0.008*"flavor" + 0.007*"price" + 0.007*"chocolate" + 0.006*"time" + 0.006*"best" + 0.005*"amazon"')]

In [100]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.037*"coffee" + 0.015*"good" + 0.012*"flavor" + 0.012*"great" + 0.012*"taste" + 0.010*"cup" + 0.008*"hot" + 0.007*"use" + 0.007*"product" + 0.007*"chocolate"'),
 (1,
  '0.017*"food" + 0.016*"tea" + 0.012*"product" + 0.011*"good" + 0.011*"great" + 0.008*"dog" + 0.007*"price" + 0.005*"time" + 0.005*"dogs" + 0.005*"treats"'),
 (2,
  '0.016*"great" + 0.015*"good" + 0.009*"taste" + 0.008*"flavor" + 0.008*"cookies" + 0.008*"chips" + 0.007*"bag" + 0.006*"snack" + 0.006*"eat" + 0.006*"delicious"')]

In [101]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.015*"sugar" + 0.012*"taste" + 0.009*"good" + 0.009*"sweet" + 0.009*"juice" + 0.007*"flavor" + 0.007*"orange" + 0.007*"water" + 0.006*"little" + 0.006*"great"'),
 (1,
  '0.015*"good" + 0.014*"chocolate" + 0.014*"flavor" + 0.012*"hot" + 0.010*"great" + 0.010*"chips" + 0.010*"taste" + 0.008*"salt" + 0.007*"oil" + 0.007*"best"'),
 (2,
  '0.022*"great" + 0.017*"product" + 0.016*"good" + 0.013*"price" + 0.010*"amazon" + 0.009*"cookies" + 0.008*"time" + 0.007*"order" + 0.007*"store" + 0.007*"free"'),
 (3,
  '0.036*"coffee" + 0.017*"tea" + 0.016*"food" + 0.012*"good" + 0.010*"great" + 0.009*"cup" + 0.008*"dog" + 0.008*"taste" + 0.008*"flavor" + 0.006*"product"')]

###### Final Model: Noun with 2 topics

In [102]:

ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=80)
ldan.print_topics()

[(0,
  '0.029*"coffee" + 0.022*"product" + 0.019*"food" + 0.012*"dog" + 0.008*"dogs" + 0.008*"price" + 0.008*"treats" + 0.008*"box" + 0.007*"order" + 0.006*"amazon"'),
 (1,
  '0.036*"taste" + 0.024*"flavor" + 0.018*"tea" + 0.016*"product" + 0.013*"chocolate" + 0.009*"water" + 0.009*"sugar" + 0.008*"ginger" + 0.007*"tastes" + 0.006*"way"')]